In [76]:
############## fusión de archivos #############

In [77]:
import pandas as pd

train_data = pd.read_csv('C:/Users/Rodrigo/OneDrive/Escritorio/UNAB/Clases_apuntes/Proyecto integradior ciencia de datos/proyecto1/dataset/train.csv')
test_data = pd.read_csv('C:/Users/Rodrigo/OneDrive/Escritorio/UNAB/Clases_apuntes/Proyecto integradior ciencia de datos/proyecto1/dataset/test.csv')

train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

data = pd.concat([train_data, test_data])

data = data.sort_values(by=['substation', 'date'])

In [78]:
############# completar fechas y horas faltantes #####################

In [79]:
all_dates = pd.date_range(start=data['date'].min(), end=data['date'].max(), freq='H')

complete_data = pd.DataFrame()

for substation in data['substation'].unique():
    substation_data = data[data['substation'] == substation]
    substation_dates = pd.DataFrame(all_dates, columns=['date'])
    substation_dates['substation'] = substation
    substation_data = pd.merge(substation_dates, substation_data, on=['substation', 'date'], how='left')
    complete_data = pd.concat([complete_data, substation_data])

complete_data['consumption'] = complete_data.groupby('substation')['consumption'].transform(lambda x: x.interpolate())

In [80]:
##################### tratamiento a outliers ######################

In [81]:
Q1 = complete_data['consumption'].quantile(0.25)
Q3 = complete_data['consumption'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

complete_data['consumption'] = complete_data['consumption'].apply(lambda x: x if lower_bound <= x <= upper_bound else None)
complete_data['consumption'] = complete_data.groupby('substation')['consumption'].transform(lambda x: x.interpolate())

In [82]:
################## Normalización ######################
from sklearn.preprocessing import StandardScaler

In [83]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

complete_data['consumption'] = scaler.fit_transform(complete_data[['consumption']])

print(complete_data['consumption'].head())

0   -0.303818
1   -0.384448
2   -0.420303
3   -0.482441
4   -0.528187
Name: consumption, dtype: float64


In [84]:
################ Entrenamiento ######################
import warnings
warnings.filterwarnings("ignore")

!pip install xgboost
!pip install lightgbm


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
train_data = complete_data[complete_data['date'] < test_data['date'].min()]
test_data = complete_data[complete_data['date'] >= test_data['date'].min()]

In [86]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

features = ['hour', 'month', 'dayofweek']

complete_data['hour'] = complete_data['date'].dt.hour
complete_data['month'] = complete_data['date'].dt.month
complete_data['dayofweek'] = complete_data['date'].dt.dayofweek

train_data = complete_data[complete_data['date'] < test_data['date'].min()]
test_data = complete_data[complete_data['date'] >= test_data['date'].min()]

X_train = train_data[features]
y_train = train_data['consumption']

models = {}

for substation in train_data['substation'].unique():
    substation_X_train = X_train[train_data['substation'] == substation]
    substation_y_train = y_train[train_data['substation'] == substation]

    X_sub_train, X_sub_val, y_sub_train, y_sub_val = train_test_split(substation_X_train, substation_y_train, test_size=0.2, random_state=42)

    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6)
    model.fit(X_sub_train, y_sub_train)

    y_pred = model.predict(X_sub_val)
    mae = mean_absolute_error(y_sub_val, y_pred)
    print(f'MAE for {substation}: {mae}')

    models[substation] = model

MAE for AJAHUEL: 0.10671562006389149
MAE for BUIN: 0.001018594141213221
MAE for CHENA: 0.10749211116288267
MAE for CNAVIA: 0.36017238526113676
MAE for ELSALTO: 0.2973594944220301
MAE for FLORIDA: 0.03139726779256665
MAE for LOSALME: 0.14160695674007884


In [87]:
################# Predicciones ##############

In [89]:
from sklearn.metrics import mean_squared_error, r2_score

# Calcular MSE y R² para cada subestación
for substation in models:
    model = models[substation]
    substation_X_test = X_test[test_data['substation'] == substation]
    substation_y_test = y_test[test_data['substation'] == substation]
    substation_predictions = model.predict(substation_X_test)
    
    mse = mean_squared_error(substation_y_test, substation_predictions)
    r2 = r2_score(substation_y_test, substation_predictions)
    
    print(f"Subestación: {substation}")
    print(f"MSE: {mse}")
    print(f"R²: {r2}")

IndexError: indices are out-of-bounds

In [90]:
################## nuevo entrenamiento normalizado por subestación #####################

In [91]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

def normalize_by_substation(substation_data):
    substation_data['consumption'] = scaler.fit_transform(substation_data[['consumption']])
    return substation_data

complete_data = complete_data.groupby('substation').apply(normalize_by_substation)

In [92]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

models = {}

for substation in train_data['substation'].unique():
    substation_X_train = X_train[train_data['substation'] == substation]
    substation_y_train = y_train[train_data['substation'] == substation]

    X_sub_train, X_sub_val, y_sub_train, y_sub_val = train_test_split(substation_X_train, substation_y_train, test_size=0.2, random_state=42)

    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6)
    model.fit(X_sub_train, y_sub_train)

    y_pred = model.predict(X_sub_val)

    mae = mean_absolute_error(y_sub_val, y_pred)
    mse = mean_squared_error(y_sub_val, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_sub_val, y_pred)
    mape = np.mean(np.abs((y_sub_val - y_pred) / y_sub_val)) * 100

    print(f'Metrics for {substation}:')
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R²: {r2}')
    print(f'MAPE: {mape}%\n')

    models[substation] = model

Metrics for AJAHUEL:
MAE: 0.10671562006389149
MSE: 0.021334514901238805
RMSE: 0.1460633934332583
R²: 0.7947689574119271
MAPE: 376.11879970267034%

Metrics for BUIN:
MAE: 0.001018594141213221
MSE: 0.00041294218404366254
RMSE: 0.020320978914502682
R²: -0.0014942833030411329
MAPE: 0.07366128676331571%

Metrics for CHENA:
MAE: 0.10749211116288267
MSE: 0.026508272165793406
RMSE: 0.16281361173376568
R²: 0.7009306031018141
MAPE: 162.91264268197193%

Metrics for CNAVIA:
MAE: 0.36017238526113676
MSE: 0.19657174463295862
RMSE: 0.44336412195052344
R²: 0.5519336490849427
MAPE: 240.6263661631184%

Metrics for ELSALTO:
MAE: 0.2973594944220301
MSE: 0.1606678495788829
RMSE: 0.40083394264817807
R²: 0.7138615366272933
MAPE: 113.52746548879516%

Metrics for FLORIDA:
MAE: 0.03139726779256665
MSE: 0.00388341317485989
RMSE: 0.06231703759695169
R²: 0.2490415779869396
MAPE: 3.0273246493977513%

Metrics for LOSALME:
MAE: 0.14160695674007884
MSE: 0.03537645999587036
RMSE: 0.18808630996399062
R²: 0.6313582027079

In [93]:
X_test = test_data[['hour', 'month', 'dayofweek']]
y_test = test_data['consumption']

predictions = []

for substation in test_data['substation'].unique():
    substation_X_test = X_test[test_data['substation'] == substation]
    model = models.get(substation)
    if model:
        substation_predictions = model.predict(substation_X_test)
        predictions.extend(substation_predictions)

predictions = np.array(predictions).reshape(-1, 1)
predictions = scaler.inverse_transform(predictions)

test_data.loc[:, 'predicted_consumption'] = predictions

print(test_data[['substation', 'date', 'consumption', 'predicted_consumption']].head())

      substation                date  consumption  predicted_consumption
35064    AJAHUEL 2022-01-01 00:00:00    -0.517680              -0.131066
35065    AJAHUEL 2022-01-01 01:00:00    -0.637328              -0.169434
35066    AJAHUEL 2022-01-01 02:00:00    -0.681343              -0.213173
35067    AJAHUEL 2022-01-01 03:00:00    -0.717677              -0.235776
35068    AJAHUEL 2022-01-01 04:00:00    -0.734051              -0.255823


In [94]:
#################### cambio de hiperparámetros ##############

In [95]:
!pip install --upgrade scikit-learn
!pip install --upgrade xgboost


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import xgboost as xgb

model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8
)

models = {}

for substation in train_data['substation'].unique():
    substation_X_train = X_train[train_data['substation'] == substation]
    substation_y_train = y_train[train_data['substation'] == substation]

    X_sub_train, X_sub_val, y_sub_train, y_sub_val = train_test_split(substation_X_train, substation_y_train, test_size=0.2, random_state=42)

    model.fit(X_sub_train, y_sub_train)

    y_pred = model.predict(X_sub_val)

    mae = mean_absolute_error(y_sub_val, y_pred)
    mse = mean_squared_error(y_sub_val, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_sub_val, y_pred)
    mape = np.mean(np.abs((y_sub_val - y_pred) / (y_sub_val + 1e-6))) * 100 
    
    print(f'Metrics for {substation}:')
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R²: {r2}')
    print(f'MAPE: {mape}%\n')

    models[substation] = model

Metrics for AJAHUEL:
MAE: 0.10628753595324142
MSE: 0.021224660200553268
RMSE: 0.14568685664998496
R²: 0.7958257236360133
MAPE: 378.2362415236959%

Metrics for BUIN:
MAE: 0.0010973938240925842
MSE: 0.0004117644919565041
RMSE: 0.020291980976644544
R²: 0.001361932260190546
MAPE: 0.08054922966968601%

Metrics for CHENA:
MAE: 0.10668760848171864
MSE: 0.026221921305279308
RMSE: 0.16193184154229615
R²: 0.7041612466767558
MAPE: 161.76133237649876%

Metrics for CNAVIA:
MAE: 0.35898347305726047
MSE: 0.19516483689369526
RMSE: 0.4417746449194377
R²: 0.5551405597118131
MAPE: 238.70331258560583%

Metrics for ELSALTO:
MAE: 0.2967522753536266
MSE: 0.1592221749482892
RMSE: 0.3990265341406373
R²: 0.7164361843767917
MAPE: 121.5668215279508%

Metrics for FLORIDA:
MAE: 0.030962903949718713
MSE: 0.0037796821782272528
RMSE: 0.061479119855665246
R²: 0.2691006502611377
MAPE: 2.985559866624595%

Metrics for LOSALME:
MAE: 0.140681442714418
MSE: 0.03503505212626317
RMSE: 0.1871765266433352
R²: 0.6349158568846597


In [97]:
################### tratamiento drástico a outliers y LightGBM ######################
!pip install lightgbm


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
### BUIN ####

In [100]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import numpy as np
import pandas as pd

train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

for df in [train_data, test_data]:
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

substation_name = 'BUIN'

def train_and_evaluate_substation_model(substation_name):
    substation_data_train = train_data[train_data['substation'] == substation_name]
    substation_data_test = test_data[test_data['substation'] == substation_name]

    if substation_data_train.empty or substation_data_test.empty:
        print(f"No hay datos para la subestación: {substation_name}")
        return

    X_train = substation_data_train[['hour', 'dayofweek', 'month', 'day']]
    y_train = substation_data_train['consumption']
    X_test = substation_data_test[['hour', 'dayofweek', 'month', 'day']]
    y_test = substation_data_test['consumption']

    model = lgb.LGBMRegressor()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-6))) * 100

    print(f"\nMétricas para la subestación {substation_name}:")
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R²: {r2}')
    print(f'MAPE: {mape}%')

substation_name = 'BUIN'
train_and_evaluate_substation_model(substation_name)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 35064, number of used features: 4
[LightGBM] [Info] Start training from score -1.145667

Métricas para la subestación BUIN:
MAE: 0.0026609995665179296
MSE: 5.281291622114936e-05
RMSE: 0.007267249563703545
R²: -2.243848254636145
MAPE: 0.23092431457522408%


In [ ]:
########### AJAHUEL ##############

In [108]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import numpy as np
import pandas as pd

substation_name = 'AJAHUEL'

def train_and_evaluate_substation_model(substation_name):
    substation_data_train = train_data[train_data['substation'] == substation_name]
    substation_data_test = test_data[test_data['substation'] == substation_name]

    if substation_data_train.empty or substation_data_test.empty:
        print(f"No hay datos para la subestación: {substation_name}")
        return

    substation_data_train['generation'] = (substation_data_train['consumption'] < 0).astype(int)
    substation_data_test['generation'] = (substation_data_test['consumption'] < 0).astype(int)

    min_train_value = substation_data_train['consumption'].min()
    if min_train_value < 0:
        substation_data_train['consumption'] += abs(min_train_value)

    min_test_value = substation_data_test['consumption'].min()
    if min_test_value < 0:
        substation_data_test['consumption'] += abs(min_test_value)

    substation_data_train['consumption'] = np.log1p(substation_data_train['consumption'])
    substation_data_test['consumption'] = np.log1p(substation_data_test['consumption'])

    X_train = substation_data_train[['hour', 'dayofweek', 'month', 'day', 'generation']]
    y_train = substation_data_train['consumption']
    X_test = substation_data_test[['hour', 'dayofweek', 'month', 'day', 'generation']]
    y_test = substation_data_test['consumption']

    model = lgb.LGBMRegressor(objective='regression', metric='mae')

    param_grid = {
        'learning_rate': [0.01, 0.1],
        'n_estimators': [100, 300, 500],
        'max_depth': [5, 7, 10],
        'num_leaves': [20, 31, 50],
        'subsample': [0.7, 0.9],
        'colsample_bytree': [0.7, 0.9],
        'min_data_in_leaf': [10, 20, 30]
    }

    def custom_mape_metric(y_true, y_pred):
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100
        return 'mape', mape, False

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=3,
        verbose=1,
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-6))) * 100

    print(f"\nMétricas para la subestación {substation_name}:")
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R²: {r2}')
    print(f'MAPE: {mape}%')

train_and_evaluate_substation_model(substation_name)

Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 35064, number of used features: 5
[LightGBM] [Info] Start training from score 1.065992
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [ ]:
def train_and_evaluate_substation_model_v3(substation_name):
    substation_data_train = train_data[train_data['substation'] == substation_name]
    substation_data_test = test_data[test_data['substation'] == substation_name]

    if substation_data_train.empty or substation_data_test.empty:
        print(f"No hay datos para la subestación: {substation_name}")
        return

    substation_data_train['is_negative'] = (substation_data_train['consumption'] < 0).astype(int)
    substation_data_test['is_negative'] = (substation_data_test['consumption'] < 0).astype(int)

    scaler = StandardScaler()
    substation_data_train['consumption_scaled'] = scaler.fit_transform(substation_data_train[['consumption']])
    substation_data_test['consumption_scaled'] = scaler.transform(substation_data_test[['consumption']])

    substation_data_train['rolling_mean'] = substation_data_train['consumption_scaled'].rolling(window=3).mean().fillna(0)
    substation_data_test['rolling_mean'] = substation_data_test['consumption_scaled'].rolling(window=3).mean().fillna(0)

    X_train = substation_data_train[['hour', 'dayofweek', 'month', 'day', 'is_negative', 'rolling_mean']]
    y_train = substation_data_train['consumption_scaled']
    X_test = substation_data_test[['hour', 'dayofweek', 'month', 'day', 'is_negative', 'rolling_mean']]
    y_test = substation_data_test['consumption_scaled']

    model = lgb.LGBMRegressor()

    param_dist = {
        'learning_rate': [0.001, 0.01, 0.1],
        'n_estimators': [100, 500, 1000],
        'max_depth': [5, 10, 15],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }

    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, cv=3, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
    random_search.fit(X_train, y_train)

    best_model = random_search.best_estimator_

    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-6))) * 100

    print(f"\nMétricas para la subestación {substation_name}:")
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R²: {r2}')
    print(f'MAPE: {mape}%')

train_and_evaluate_substation_model_v3(substation_name)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [103]:
print("Subestaciones en el conjunto de entrenamiento:", train_data['substation'].unique())
print("Subestaciones en el conjunto de prueba:", test_data['substation'].unique())

Subestaciones en el conjunto de entrenamiento: ['AJAHUEL' 'BUIN' 'CHENA' 'CNAVIA' 'ELSALTO' 'FLORIDA' 'LOSALME']
Subestaciones en el conjunto de prueba: ['AJAHUEL' 'BUIN' 'CHENA' 'CNAVIA' 'ELSALTO' 'FLORIDA' 'LOSALME']
